In [2]:
%load_ext autoreload
%autoreload 2


In [3]:
from smooth.adam_general.core.adam import Adam

import numpy as np
import pandas as pd

In [4]:
from typing import List, Union, Dict, Any
from smooth.adam_general._adam_general import adam_fitter, adam_forecaster
from numpy.linalg import eigvals
import nlopt

In [5]:
#### Global level and ETS(A,N,N) ####
np.random.seed(33)  # For reproducibility
n_points = 100
time_series = np.random.normal(100, 10, n_points)  # Random errors
ts_df = pd.DataFrame({'value': time_series}, index=pd.date_range(start='2023-01-01', periods=n_points, freq='ME'))

In [6]:
adam = Adam(model="ANN", lags=[12])
adam.fit(ts_df, h = 12)
adam.predict()

,mean,lower_0.025,upper_0.975
2031-05-31,99.619575,NaN,NaN
2031-06-30,99.619575,NaN,NaN
2031-07-31,99.619575,NaN,NaN
2031-08-31,99.619575,NaN,NaN
2031-09-30,99.619575,NaN,NaN
2031-10-31,99.619575,NaN,NaN
2031-11-30,99.619575,NaN,NaN
2031-12-31,99.619575,NaN,NaN
2032-01-31,99.619575,NaN,NaN
2032-02-29,99.619575,NaN,NaN


In [7]:
#### Local level and ETS(A,N,N) ####
np.random.seed(41)  # For reproducibility
n_points = 120
errors = np.random.normal(0, 10, n_points)  # Random errors
time_series = np.zeros(n_points)
time_series[0] = 100
for i in range(n_points-1):
    # This is SES with alpha=0.1, just in an alternative form
    time_series[i+1] = time_series[i] + (0.1-1) * errors[i] + errors[i+1]
ts_df = pd.DataFrame({'value': time_series}, index=pd.date_range(start='2023-01-01', periods=n_points, freq='ME'))

In [8]:
# ETS(A,N,N) with alpha=0.1
model = Adam(model="ANN", lags=[12], persistence=0.1)
model.fit(ts_df, h=12)
print("ETS(A,N,N) with alpha=0.1:")
print(model.predict())

# ETS(A,N,N) with estimated alpha 
model = Adam(model="ANN", lags=[12])
model.fit(ts_df, h=12)
print("\nETS(A,N,N) with estimated parameters:")
print(model.adam_estimated["B"])
print(model.predict())

# ETS(M,N,N) with estimated alpha
model = Adam(model="MNN", lags=[12], distribution="dnorm")
model.fit(ts_df, h=12)
print("\nETS(M,N,N) with estimated parameters:")
print(model.predict())


ETS(A,N,N) with alpha=0.1:
                 mean  lower_0.025  upper_0.975
2033-01-31  83.225986          NaN          NaN
2033-02-28  83.225986          NaN          NaN
2033-03-31  83.225986          NaN          NaN
2033-04-30  83.225986          NaN          NaN
2033-05-31  83.225986          NaN          NaN
2033-06-30  83.225986          NaN          NaN
2033-07-31  83.225986          NaN          NaN
2033-08-31  83.225986          NaN          NaN
2033-09-30  83.225986          NaN          NaN
2033-10-31  83.225986          NaN          NaN
2033-11-30  83.225986          NaN          NaN
2033-12-31  83.225986          NaN          NaN

ETS(A,N,N) with estimated parameters:
[6.53178667e-02 9.69906981e+01]
                 mean  lower_0.025  upper_0.975
2033-01-31  84.708881          NaN          NaN
2033-02-28  84.708881          NaN          NaN
2033-03-31  84.708881          NaN          NaN
2033-04-30  84.708881          NaN          NaN
2033-05-31  84.708881          NaN    

In [9]:
#### Local trend and ETS(A,A,N) ####
np.random.seed(42)  # For reproducibility
n_points = 120
errors = np.random.normal(0, 10, n_points)  # Random errors
trend = np.random.normal(0.5, 2, n_points)  # Random trend
time_series = np.zeros(n_points)
time_series[0] = 100
for i in range(n_points-1):
    # This is SES with alpha=0.1, just in an alternative form
    time_series[i+1] = time_series[i] + (0.1-1) * errors[i] + trend[i] + errors[i+1]
ts_df = pd.DataFrame({'value': time_series}, index=pd.date_range(start='2023-01-01', periods=n_points, freq='ME'))


In [10]:
# ETS(A,A,N) with fixed parameters
model = Adam(model="AAN", lags=[12], persistence=[0.1, 0.02])
model.fit(ts_df, h=12)
print("ETS(A,A,N):")
print(model.predict())

# ETS(A,Ad,N) with estimated parameters
model = Adam(model="AAdN", lags=[12])
model.fit(ts_df, h=12)
print("\nETS(A,Ad,N) estimated parameters:")
print(model.adam_estimated["B"])

# ETS(A,Ad,N) with fixed parameters
model = Adam(model="AAdN", lags=[12], persistence=[0.1, 0.02])
model.fit(ts_df, h=12)
print("\nETS(A,Ad,N):")
print(model.predict())

ETS(A,A,N):
                  mean  lower_0.025  upper_0.975
2033-01-31  165.255525          NaN          NaN
2033-02-28  165.763955          NaN          NaN
2033-03-31  166.272384          NaN          NaN
2033-04-30  166.780814          NaN          NaN
2033-05-31  167.289244          NaN          NaN
2033-06-30  167.797674          NaN          NaN
2033-07-31  168.306104          NaN          NaN
2033-08-31  168.814534          NaN          NaN
2033-09-30  169.322963          NaN          NaN
2033-10-31  169.831393          NaN          NaN
2033-11-30  170.339823          NaN          NaN
2033-12-31  170.848253          NaN          NaN

ETS(A,Ad,N) estimated parameters:
[ 1.68275229e-01  1.04248642e-02  9.86301308e-01  1.04774858e+02
 -3.80921118e-01]

ETS(A,Ad,N):
                  mean  lower_0.025  upper_0.975
2033-01-31  163.235667          NaN          NaN
2033-02-28  163.701313          NaN          NaN
2033-03-31  164.140767          NaN          NaN
2033-04-30  164.555501 

In [11]:
# ETS(M,A,N) with fixed parameters
model = Adam(model="MAN", lags=[12], persistence=[0.1, 0.02], distribution="dnorm")
model.fit(ts_df, h=12)
print("ETS(M,A,N):")
print(model.predict())

# ETS(M,Ad,N) with estimated parameters
model = Adam(model="MAdN", lags=[12], distribution="dnorm")
model.fit(ts_df, h=12)
print("\nETS(M,Ad,N) estimated parameters:")
print(model.adam_estimated["B"])

# ETS(M,Ad,N) with fixed parameters
model = Adam(model="MAdN", lags=[12], persistence=[0.1, 0.02], distribution="dnorm")
model.fit(ts_df, h=12)
print("\nETS(M,Ad,N):")
print(model.predict())

ETS(M,A,N):
                  mean  lower_0.025  upper_0.975
2033-01-31  165.256254          NaN          NaN
2033-02-28  165.764546          NaN          NaN
2033-03-31  166.272837          NaN          NaN
2033-04-30  166.781129          NaN          NaN
2033-05-31  167.289421          NaN          NaN
2033-06-30  167.797713          NaN          NaN
2033-07-31  168.306005          NaN          NaN
2033-08-31  168.814297          NaN          NaN
2033-09-30  169.322588          NaN          NaN
2033-10-31  169.830880          NaN          NaN
2033-11-30  170.339172          NaN          NaN
2033-12-31  170.847464          NaN          NaN

ETS(M,Ad,N) estimated parameters:
[ 1.49690928e-01  1.32751821e-02  9.67541206e-01  1.05756209e+02
 -4.62346860e-01]

ETS(M,Ad,N):
                  mean  lower_0.025  upper_0.975
2033-01-31  163.195224          NaN          NaN
2033-02-28  163.659519          NaN          NaN
2033-03-31  164.096847          NaN          NaN
2033-04-30  164.508774 

In [13]:
# ETS(M,M,N) with fixed parameters
model = Adam(model="MMN", lags=[12], persistence=[0.1, 0.02], distribution="dnorm")
model.fit(ts_df, h=12)
print("ETS(M,M,N):")
print(model.adam_estimated["B"])
print(model.predict())

# ETS(M,Md,N) with estimated parameters
model = Adam(model="MMdN", lags=[12], distribution="dnorm")
model.fit(ts_df, h=12)
print("\nETS(M,Md,N) estimated parameters:")
print(model.adam_estimated["B"])

# ETS(M,Md,N) with fixed parameters
model = Adam(model="MMdN", lags=[12], persistence=[0.1, 0.02], distribution="dnorm")
model.fit(ts_df, h=12)
print("\nETS(M,Md,N):")
print(model.predict())


ETS(M,M,N):
[107.91006209   0.9932759 ]
                  mean  lower_0.025  upper_0.975
2033-01-31  165.593071          NaN          NaN
2033-02-28  166.074017          NaN          NaN
2033-03-31  166.556360          NaN          NaN
2033-04-30  167.040104          NaN          NaN
2033-05-31  167.525253          NaN          NaN
2033-06-30  168.011811          NaN          NaN
2033-07-31  168.499782          NaN          NaN
2033-08-31  168.989170          NaN          NaN
2033-09-30  169.479980          NaN          NaN
2033-10-31  169.972215          NaN          NaN
2033-11-30  170.465880          NaN          NaN
2033-12-31  170.960978          NaN          NaN


/home/isvetunkov/Cursor/smooth/python/smooth/adam_general/core/forecaster.py:134: UserWarning: Your model has a potentially explosive multiplicative trend. I cannot do anything about it, so please just be careful.
  warnings.warn("Your model has a potentially explosive multiplicative trend. "



ETS(M,Md,N) estimated parameters:
[1.49082653e-01 1.27970575e-02 9.65242357e-01 1.06257974e+02
 9.94932936e-01]

ETS(M,Md,N):
                  mean  lower_0.025  upper_0.975
2033-01-31  163.174599          NaN          NaN
2033-02-28  163.629296          NaN          NaN
2033-03-31  164.056585          NaN          NaN
2033-04-30  164.458048          NaN          NaN
2033-05-31  164.835188          NaN          NaN
2033-06-30  165.189425          NaN          NaN
2033-07-31  165.522103          NaN          NaN
2033-08-31  165.834495          NaN          NaN
2033-09-30  166.127801          NaN          NaN
2033-10-31  166.403155          NaN          NaN
2033-11-30  166.661630          NaN          NaN
2033-12-31  166.904234          NaN          NaN
